# WEBmedSTAT: визуализация медицинской статистики
---
**Цель проекта:** превращение разрозненных таблиц статистики в простой и наглядный дашборд.

**Данные** были получены путем парсинга сайта Росстата. Данные являются общедоступными, их использование в любых коммерческих и некоммерческих проектах разрешено.

**Тема для построения дашборда:** демографическая ситуация.

**Гипотезы для исследования:** подтверждаются ли предположения о том, что за период 2001 - 2023:

- Общая численность населения страны снижается.

- Население в своей массе становится более старым.

- Население умирает в более молодом возрасте.

- Женщины рожают в более старшем возрасте.

- Соотношение мертворожденных мужского и женского пола не имеет статистически значимых отличий, в то время как соотношение перинатально умерших (первые 168 часов, 7 суток после родов) мальчиков и девочек всегда (из года в год) выше у мальчиков. Действительно ли это так?

#### **Ссылка на дашборд:** https://datalens.yandex/oa9gdxk4re3gc



## 1. Предобработка данных

In [1]:
# Импорт библиотек, настройка вывода данных

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats as st
from statsmodels.stats.proportion import proportions_ztest
import math as mth

%matplotlib inline

pd.set_option('display.max_columns', 60)

### 1.1. Общая численность и возрастные группы населения

---
Сведения об общей численности населения по годам содержатся в нескольких таблицах (показатели демографической ситуации).

В них же есть сведения о численности населения по годам по возрастным группам 'моложе трудоспособного', 'трудоспособное' и 'старше трудоспособного' - эти показатели можно использовать для анализа того, становится ли население более старым.

Вместе с тем, в таблицах много лишней для анализа информации, которую стоит убрать.

---

In [2]:
# Создание функции для обработки датафреймов, содержащих сведения об общей численности и возрастных группах населения по годам. Функция принимает на вход исходный датафрейм, список индексов строк, которые нужно оставить в итоговом датафрейме, и список годов (поскольку исходные датафреймы сформированы по-разному, проще задавать этот список вручную)

def headcount_func(df, rows_to_keep, years_list):
    df = df.iloc[rows_to_keep].T
    df.columns = ['год', 'все население', 'моложе трудоспособного', 'трудоспособное', 'старше трудоспособного']
    df = df.drop(df.index[0]).reset_index(drop=True)
    df['год'] = years_list
    return df

In [3]:
# Запись исходного датафрейма в переменную

headcount2005 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2005\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\показатели демографической ситуации.csv")

In [4]:
headcount2005

,0,1,2,3,4,5,6,7,8
0,NaN,1990.0,1995.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0
1,"Численность населения1), тыс. человек:",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,на начало года,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,все население,147665.0,148460.0,146890.0,146304.0,145649.0,144964.0,144168.0,143474.0
4,городское,108736.0,108322.0,107419.0,107072.0,106725.0,106321.0,105818.0,104719.0
5,сельское,38929.0,40138.0,39471.0,39232.0,38924.0,38643.0,38350.0,38755.0
6,в среднем за год,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,все население,147969.0,148376.0,146597.0,145977.0,145306.0,144566.0,143821.0,NaN
8,городское,109070.0,108316.0,107246.0,106899.0,106523.0,106070.0,105269.0,NaN
9,сельское,38899.0,40060.0,39351.0,39078.0,38783.0,38496.0,38552.0,NaN


In [5]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2005 = [0, 3, 21, 22, 23]
years_list2005 = [1990, 1995, 2000, 2001, 2002, 2003, 2004, 2005]
headcount2005 = headcount_func(headcount2005, rows_to_keep2005, years_list2005)

In [6]:
# Демонстрация измененного датафрейма

headcount2005

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,1990,147665.0,36101.0,83943.0,27621.0
1,1995,148460.0,34252.0,84332.0,29876.0
2,2000,146890.0,29580.0,87172.0,30138.0
3,2001,146304.0,28387.0,88040.0,29877.0
4,2002,145649.0,27274.0,88515.0,29860.0
5,2003,144964.0,26115.0,89206.0,29643.0
6,2004,144168.0,25014.0,89896.0,29258.0
7,2005,143474.0,24095.0,90218.0,29161.0


In [7]:
# Запись исходного датафрейма в переменную

headcount2007 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2007\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\показатели демографической ситуации.csv")

In [8]:
headcount2007.head()

,0,1,2,3,4,5
0,NaN,1990.0,1995.0,2000.0,2005.0,2006.0
1,"Численность населения, тыс. человек:",NaN,NaN,NaN,NaN,NaN
2,на конец года,NaN,NaN,NaN,NaN,NaN
3,все население,148274.0,148292.0,146304.0,142754.0,142221.0
4,городское,109405.0,108311.0,107072.0,104105.0,103778.0


In [9]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2007 = [0, 3, 21, 22, 23]
years_list2007 = [1990, 1995, 2000, 2005, 2006]
headcount2007 = headcount_func(headcount2007, rows_to_keep2007, years_list2007)

In [10]:
# Демонстрация измененного датафрейма

headcount2007

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,1990,148274.0,36053.0,84048.0,28173.0
1,1995,148292.0,33615.0,84540.0,30137.0
2,2000,146304.0,28387.0,88040.0,29877.0
3,2005,142754.0,23317.0,90328.0,29109.0
4,2006,142221.0,22718.0,90152.0,29351.0


In [11]:
# Запись исходного датафрейма в переменную

headcount2009 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2009\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\показатели демографической ситуации.csv")

In [12]:
headcount2009.head()

,0,1,2,3,4,5
0,NaN,1995.0,2000.0,2005.0,2007.0,2008.0
1,"Численность населения, тыс. человек:",NaN,NaN,NaN,NaN,NaN
2,на конец года,NaN,NaN,NaN,NaN,NaN
3,все население,148292.0,146304.0,142754.0,142009.0,141904.0
4,городское,108311.0,107072.0,104105.0,103773.0,103690.0


In [13]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2009 = [0, 3, 21, 22, 23]
years_list2009 = [1995, 2000, 2005, 2007, 2008]
headcount2009 = headcount_func(headcount2009, rows_to_keep2009, years_list2009)

In [14]:
# Демонстрация измененного датафрейма

headcount2009

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,1995,148292.0,33615.0,84540.0,30137.0
1,2000,146304.0,28387.0,88040.0,29877.0
2,2005,142754.0,23317.0,90328.0,29109.0
3,2007,142009.0,22497.0,89752.0,29760.0
4,2008,141904.0,22541.0,89266.0,30097.0


In [15]:
# Запись исходного датафрейма в переменную

headcount2013 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2013\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.1. показатели демографической ситуации.csv")

In [16]:
headcount2013.head()

,Unnamed: 0,2000,2005,2010,2011,2012
0,"Численность населения, тыс. человек:",NaN,NaN,NaN,NaN,NaN
1,на конец года,NaN,NaN,NaN,NaN,NaN
2,все население,146304,143236,142865,143056,143347
3,городское,107072,104818,105421,105742,106118
4,сельское,39232,38418,37444,37314,37229


In [17]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2013 = [0, 2, 20, 21, 22]
years_list2013 = [2000, 2005, 2010, 2011, 2012]
headcount2013 = headcount_func(headcount2013, rows_to_keep2013, years_list2013)

In [18]:
# Демонстрация измененного датафрейма

headcount2013

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,2000,146304,28387,88040,29877
1,2005,143236,23671,90157,29408
2,2010,142865,23209,87847,31809
3,2011,143056,23568,87055,32433
4,2012,143347,24110,86137,33100


In [19]:
# Запись исходного датафрейма в переменную

headcount2015 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2015\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.1. показатели демографической ситуации.csv")

In [20]:
headcount2015.head()

,Unnamed: 0,2005,2010,2011,2012,2013,2014
0,"Численность населения, тыс. человек:",NaN,NaN,NaN,NaN,NaN,NaN
1,на конец года,NaN,NaN,NaN,NaN,NaN,NaN
2,все население,143236,142865,143056,143347,143667,146267
3,городское,104818,105421,105742,106118,106549,108282
4,сельское,38418,37444,37314,37229,37118,37985


In [21]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2015 = [0, 2, 20, 21, 22]
years_list2015 = [2005, 2010, 2011, 2012, 2013, 2014]
headcount2015 = headcount_func(headcount2015, rows_to_keep2015, years_list2015)

In [22]:
# Демонстрация измененного датафрейма

headcount2015

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,2005,143236,23671,90157,29408
1,2010,142865,23209,87847,31809
2,2011,143056,23568,87055,32433
3,2012,143347,24110,86137,33100
4,2013,143667,24717,85162,33788
5,2014,146267,25689,85415,35163


In [23]:
# Запись исходного датафрейма в переменную

headcount2017 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2017\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.1. показатели демографической ситуации.csv")

In [24]:
headcount2017.head()

,Unnamed: 0,2005,2010,2013,2014,2015,2016
0,"Численность населения, тыс. человек:",NaN,NaN,NaN,NaN,NaN,NaN
1,на конец года,NaN,NaN,NaN,NaN,NaN,NaN
2,все население,143236,142865,143667,146267,146545,146804
3,городское,104818,105421,106549,108282,108658,109032
4,сельское,38418,37444,37118,37985,37887,37772


In [25]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2017 = [0, 2, 20, 21, 22]
years_list2017 = [2005, 2010, 2013, 2014, 2015, 2016]
headcount2017 = headcount_func(headcount2017, rows_to_keep2017, years_list2017)

In [26]:
# Демонстрация измененного датафрейма

headcount2017

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,2005,143236,23671,90157,29408
1,2010,142865,23209,87847,31809
2,2013,143667,24717,85162,33788
3,2014,146267,25689,85415,35163
4,2015,146545,26360,84199,35986
5,2016,146804,26895,83224,36685


In [27]:
# Запись исходного датафрейма в переменную

headcount2019 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2019\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.1. показатели демографической ситуации.csv")

In [28]:
headcount2019.head()

,Unnamed: 0,2005,2010,2015,2016,2017,2018
0,"Численность населения, тыс. человек:",NaN,NaN,NaN,NaN,NaN,NaN
1,на конец года,NaN,NaN,NaN,NaN,NaN,NaN
2,все население,143236,142865,146545,146804,146880,146781
3,городское,104818,105421,108658,109032,109327,109454
4,сельское,38418,37444,37887,37772,37553,37327


In [29]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2019 = [0, 2, 20, 21, 22]
years_list2019 = [2005, 2010, 2015, 2016, 2017, 2018]
headcount2019 = headcount_func(headcount2019, rows_to_keep2019, years_list2019)

In [30]:
# Демонстрация измененного датафрейма

headcount2019

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,2005,143236,23671,90157,29408
1,2010,142865,23209,87847,31809
2,2015,146545,26360,84199,35986
3,2016,146804,26895,83224,36685
4,2017,146880,27254,82264,37362
5,2018,146781,27430,81362,37989


In [31]:
# Запись исходного датафрейма в переменную

headcount2021 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2021\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.1. показатели демографической ситуации.csv")

In [32]:
headcount2021.head()

,Unnamed: 0,2005,2010,2015,2018,2019,2020
0,"Численность населения, тыс. человек:",NaN,NaN,NaN,NaN,NaN,NaN
1,на конец года,NaN,NaN,NaN,NaN,NaN,NaN
2,все население,143 236,142 865,146 545,146 781,146 749,146 171
3,городское,104 818,105 421,108 658,109 454,109 563,109 252
4,сельское,38 418,37 444,37 887,37 327,37 186,36 919


In [33]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2021 = [0, 2, 20, 21, 22]
years_list2021 = [2005, 2010, 2015, 2018, 2019, 2020]
headcount2021 = headcount_func(headcount2021, rows_to_keep2021, years_list2021)

In [34]:
# Демонстрация измененного датафрейма

headcount2021

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,2005,143 236,23 671,90 157,29 408
1,2010,142 865,23 209,87 847,31 809
2,2015,146 545,26 360,84 199,35 986
3,2018,146 781,27 430,81 362,37 989
4,2019,146 749,27 442,82 678,36 629
5,2020,146 171,27 387,81 881,36 903


In [35]:
# Запись исходного датафрейма в переменную

headcount2023 = pd.read_excel("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2023\\Zdrav_01-2023.xlsx", sheet_name='1.1')

In [36]:
headcount2023.head()

,1.1. ПОКАЗАТЕЛИ ДЕМОГРАФИЧЕСКОЙ СИТУАЦИИ,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,2010,2015.0,2019.0,2020.0,2021.0,2022.0
1,"Численность населения1), тыс. человек:",NaN,NaN,NaN,NaN,NaN,NaN
2,на конец года,NaN,NaN,NaN,NaN,NaN,NaN
3,все население,142 865,147182.0,147959.0,147456.0,146980.0,146447.0
4,городское,105 421,109315.0,110542.0,110221.0,109982.0,109655.0


In [37]:
# Применение функции для обработки исходного датафрейма

rows_to_keep2023 = [0, 3, 21, 22, 23]
years_list2023 = [2010, 2015, 2019, 2020, 2021, 2022]
headcount2023 = headcount_func(headcount2023, rows_to_keep2023, years_list2023)

In [38]:
# Демонстрация измененного датафрейма

headcount2023

,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,2010,142 865,23 209,87 847,31 809
1,2015,147182.0,26480.0,84657.0,36045.0
2,2019,147959.0,27440.0,83679.0,36840.0
3,2020,147456.0,27385.0,82926.0,37145.0
4,2021,146980.0,27346.0,83663.0,36209.0
5,2022,146447.0,27160.0,83440.0,35847.0


---
Объединяем все измененные датасеты в один.

Поскольку в датасетах много раз повторяются одни и те же года, причем интересующие нас показатели за один и тот же год могут отличаться в разных датасетах, целесообразно удалить дублирующиеся годы, причем оставить записи из более "поздних" датасетов (предполагается, что со временем данные актуализируются и более новые - более актуальные).

---

In [39]:
# Объединение датасетов и удаление дубликатов

headcount_combined = (
    pd.concat([headcount2005, headcount2007, headcount2009, headcount2013, headcount2015, headcount2017, headcount2019, headcount2021, headcount2023], axis=0)
#    .drop_duplicates(subset='год', keep='last')
    .sort_values(by='год')
    .query('год >= 2001')
    .reset_index(drop=True)
)

In [40]:
# Удаление лишних символов (неразрывные пробелы, обнаружившиеся при попытке изменения типа данных), и изменение типа данных на числовой

headcount_combined[['все население', 'моложе трудоспособного', 'трудоспособное', 'старше трудоспособного']] = (
    headcount_combined[['все население', 'моложе трудоспособного', 'трудоспособное', 'старше трудоспособного']]
    .apply(lambda x: x.astype(str).str.replace('\xa0', '', regex=True)
           .astype(float), axis=0)
)

In [41]:
# Вывод информации о скомпилированном датафрейме

headcount_combined.info()
headcount_combined.head(44)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   год                     44 non-null     int64  
 1   все население           44 non-null     float64
 2   моложе трудоспособного  44 non-null     float64
 3   трудоспособное          44 non-null     float64
 4   старше трудоспособного  44 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 1.8 KB


,год,все население,моложе трудоспособного,трудоспособное,старше трудоспособного
0,2001,146304.0,28387.0,88040.0,29877.0
1,2002,145649.0,27274.0,88515.0,29860.0
2,2003,144964.0,26115.0,89206.0,29643.0
3,2004,144168.0,25014.0,89896.0,29258.0
4,2005,143236.0,23671.0,90157.0,29408.0
5,2005,143236.0,23671.0,90157.0,29408.0
6,2005,143236.0,23671.0,90157.0,29408.0
7,2005,143236.0,23671.0,90157.0,29408.0
8,2005,142754.0,23317.0,90328.0,29109.0
9,2005,142754.0,23317.0,90328.0,29109.0


In [42]:
# Запись скомпилированного датафрейма в файл

headcount_combined.to_csv('headcount_combined.csv', encoding='utf-8')

In [43]:
headcount_combined_melted = headcount_combined.melt(id_vars=['год', 'все население'], var_name='категория', value_name='значение')

In [44]:
headcount_combined_melted

,год,все население,категория,значение
0,2001,146304.0,моложе трудоспособного,28387.0
1,2002,145649.0,моложе трудоспособного,27274.0
2,2003,144964.0,моложе трудоспособного,26115.0
3,2004,144168.0,моложе трудоспособного,25014.0
4,2005,143236.0,моложе трудоспособного,23671.0
...,...,...,...,...
127,2019,146749.0,старше трудоспособного,36629.0
128,2020,146171.0,старше трудоспособного,36903.0
129,2020,147456.0,старше трудоспособного,37145.0
130,2021,146980.0,старше трудоспособного,36209.0


In [45]:
headcount_combined_melted.to_csv('headcount_combined_melted.csv', encoding='utf-8')

### 1.2. Смертность по возрастным группам

---
Сведения о смертности населения по годам, по полу и возрастным группам, содержатся в нескольких таблицах (возрастные коэффициенты смертности).

В таблицах много лишней для анализа информации, которую стоит убрать.

---

In [46]:
# Создание функции для обработки датафреймов, содержащих сведения о смертности населения по годам, по полу и возрастным группам. Функция принимает на вход исходный датафрейм, список индексов строк с возрастными группами мужчин и женщин, которые нужно оставить в итоговом датафрейме, и список годов (поскольку исходные датафреймы сформированы по-разному, проще задавать этот список вручную)

def mortality_age_func(df, years_list, men_range, women_range):
    years = years_list
    years.insert(0, 'возраст')
    years.append('пол')
    mortality_age_men = df.iloc[men_range]
    mortality_age_men.loc[:, ['пол']] = 'мужчины'
    mortality_age_women = df.iloc[women_range]
    mortality_age_women.loc[:, ['пол']] = 'женщины'
    mortality_age = pd.concat([mortality_age_men, mortality_age_women]).reset_index(drop=True)
    mortality_age.columns = years
    return mortality_age

In [47]:
# Запись исходного датафрейма в переменную

mortality_age2005 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2005\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\возрастные коэффициенты смертности.csv")
mortality_age2005

,0,1,2,3,4,5,6,7
0,NaN,1990.0,1995.0,2000.0,2001.0,2002.0,2003.0,2004.0
1,Мужчины,11.6,16.9,17.3,17.8,18.6,18.9,18.6
2,из них:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"в возрасте, лет:",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,20.0,20.5,17.3,16.9,15.3,13.8,13.1
5,1-4,1.1,1.2,1.1,1.1,1.0,1.0,0.8
6,5-9,0.7,0.7,0.6,0.6,0.5,0.5,0.5
7,10-14,0.6,0.7,0.6,0.6,0.6,0.6,0.6
8,15-19,1.6,2.4,2.1,1.9,1.8,1.7,1.7
9,20-24,2.6,4.4,4.9,4.4,4.0,3.9,3.9


In [48]:
# Применение функции для обработки исходного датафрейма

years_list2005 = [1990, 1995, 2000, 2001, 2002, 2003, 2004]
men_range2005 = slice(4, 20)
women_range2005 = slice(24, 40)

mortality_age2005 = mortality_age_func(mortality_age2005, years_list2005, men_range2005, women_range2005)

In [49]:
# Демонстрация измененного датафрейма

mortality_age2005

,возраст,1990,1995,2000,2001,2002,2003,2004,пол
0,0,20.0,20.5,17.3,16.9,15.3,13.8,13.1,мужчины
1,1-4,1.1,1.2,1.1,1.1,1.0,1.0,0.8,мужчины
2,5-9,0.7,0.7,0.6,0.6,0.5,0.5,0.5,мужчины
3,10-14,0.6,0.7,0.6,0.6,0.6,0.6,0.6,мужчины
4,15-19,1.6,2.4,2.1,1.9,1.8,1.7,1.7,мужчины
5,20-24,2.6,4.4,4.9,4.4,4.0,3.9,3.9,мужчины
6,25-29,3.3,5.6,6.0,5.8,5.7,5.9,6.1,мужчины
7,30-34,4.3,7.4,7.0,7.1,7.3,7.5,7.8,мужчины
8,35-39,5.6,10.1,9.1,9.4,9.9,10.2,10.2,мужчины
9,40-44,7.7,14.3,12.7,13.1,13.9,14.4,14.2,мужчины


In [50]:
# Запись исходного датафрейма в переменную

mortality_age2007 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2007\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\возрастные коэффициенты смертности.csv")
mortality_age2007.head()

,0,1,2,3,4,5
0,NaN,1990.0,1995.0,2000.0,2005.0,2006.0
1,Мужчины,11.6,16.9,17.3,18.8,17.4
2,из них:,NaN,NaN,NaN,NaN,NaN
3,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN
4,0,20.0,20.5,17.3,12.5,11.4


In [51]:
# Применение функции для обработки исходного датафрейма

years_list2007 = [1990, 1995, 2000, 2005, 2006]
men_range2007 = slice(4, 20)
women_range2007 = slice(24, 40)

mortality_age2007 = mortality_age_func(mortality_age2007, years_list2007, men_range2007, women_range2007)

In [52]:
# Демонстрация измененного датафрейма

mortality_age2007.head()

,возраст,1990,1995,2000,2005,2006,пол
0,0,20.0,20.5,17.3,12.5,11.4,мужчины
1,1-4,1.1,1.2,1.1,0.8,0.8,мужчины
2,5-9,0.7,0.7,0.6,0.5,0.4,мужчины
3,10-14,0.6,0.7,0.6,0.5,0.5,мужчины
4,15-19,1.6,2.4,2.1,1.6,1.6,мужчины


In [53]:
# Запись исходного датафрейма в переменную

mortality_age2009 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2009\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\возрастные коэффициенты смертности.csv")
mortality_age2009.head()

,0,1,2,3,4,5
0,NaN,1995.0,2000.0,2005.0,2007.0,2008.0
1,Мужчины,16.9,17.3,18.8,16.7,16.6
2,из них:,NaN,NaN,NaN,NaN,NaN
3,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN
4,0,20.5,17.3,12.5,10.5,9.5


In [54]:
# Применение функции для обработки исходного датафрейма

years_list2009 = [1995, 2000, 2005, 2007, 2008]
men_range2009 = slice(4, 20)
women_range2009 = slice(24, 40)

mortality_age2009 = mortality_age_func(mortality_age2009, years_list2009, men_range2009, women_range2009)

In [55]:
# Демонстрация измененного датафрейма

mortality_age2009.head()

,возраст,1995,2000,2005,2007,2008,пол
0,0,20.5,17.3,12.5,10.5,9.5,мужчины
1,1-4,1.2,1.1,0.8,0.7,0.7,мужчины
2,5-9,0.7,0.6,0.5,0.4,0.4,мужчины
3,10-14,0.7,0.6,0.5,0.5,0.4,мужчины
4,15-19,2.4,2.1,1.6,1.5,1.5,мужчины


In [56]:
# Запись исходного датафрейма в переменную

mortality_age2013 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2013\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.5. возрастные коэффициенты смертности (умершие на 1000 человек населения соответствующей возрастной группы).csv")
mortality_age2013.head()

,Unnamed: 0,2000,2005,2010,2011,2012
0,Мужчины,"17,3","18,7","15,9","15,1","14,7"
1,из них:,NaN,NaN,NaN,NaN,NaN
2,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN
3,0,"17,3","12,5","8,3","8,3","9,5"
4,1-4,"1,1","0,8","0,6","0,6","0,5"


In [57]:
# Применение функции для обработки исходного датафрейма

years_list2013 = [2000, 2005, 2010, 2011, 2012]
men_range2013 = slice(3, 19)
women_range2013 = slice(23, 39)

mortality_age2013 = mortality_age_func(mortality_age2013, years_list2013, men_range2013, women_range2013)

In [58]:
# Демонстрация измененного датафрейма

mortality_age2013.head()

,возраст,2000,2005,2010,2011,2012,пол
0,0,"17,3","12,5","8,3","8,3","9,5",мужчины
1,1-4,"1,1","0,8","0,6","0,6","0,5",мужчины
2,5-9,"0,6","0,5","0,4","0,3","0,3",мужчины
3,10-14,"0,6","0,5","0,4","0,4","0,4",мужчины
4,15-19,"2,1","1,6","1,2","1,2","1,1",мужчины


In [59]:
# Запись исходного датафрейма в переменную

mortality_age2015 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2015\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.5. возрастные коэффициенты смертности (умершие на 1000 человек населения соответствующей возрастной группы).csv")
mortality_age2015.head()

,Unnamed: 0,2005,2010,2011,2012,2013,2014
0,Мужчины,"18,7","15,9","15,1","14,7","14,4","14,4"
1,из них:,NaN,NaN,NaN,NaN,NaN,NaN
2,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN,NaN
3,0,"12,5","8,3","8,3","9,5","9,0","8,1"
4,1-4,"0,8","0,6","0,6","0,5","0,5","0,5"


In [60]:
# Применение функции для обработки исходного датафрейма

years_list2015 = [2005, 2010, 2011, 2012, 2013, 2014]
men_range2015 = slice(3, 19)
women_range2015 = slice(23, 39)

mortality_age2015 = mortality_age_func(mortality_age2015, years_list2015, men_range2015, women_range2015)

In [61]:
# Демонстрация измененного датафрейма

mortality_age2015.head()

,возраст,2005,2010,2011,2012,2013,2014,пол
0,0,"12,5","8,3","8,3","9,5","9,0","8,1",мужчины
1,1-4,"0,8","0,6","0,6","0,5","0,5","0,5",мужчины
2,5-9,"0,5","0,4","0,3","0,3","0,3","0,3",мужчины
3,10-14,"0,5","0,4","0,4","0,4","0,4","0,4",мужчины
4,15-19,"1,6","1,2","1,2","1,1","1,1","1,1",мужчины


In [62]:
# Запись исходного датафрейма в переменную

mortality_age2017 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2017\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.5. возрастные коэффициенты смертности (умершие на 1000 человек населения соответствующей возрастной группы).csv")
mortality_age2017.head()

,Unnamed: 0,2005,2010,2013,2014,2015,2016
0,Мужчины,"18,7","15,9","14,4","14,4","14,2","14,0"
1,из них:,NaN,NaN,NaN,NaN,NaN,NaN
2,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN,NaN
3,01),"12,5","8,6","9,0","8,1","7,3","6,6"
4,1-4,"0,8","0,6","0,5","0,5","0,4","0,4"


In [63]:
# Применение функции для обработки исходного датафрейма

years_list2017 = [2005, 2010, 2013, 2014, 2015, 2016]
men_range2017 = slice(3, 19)
women_range2017 = slice(23, 39)

mortality_age2017 = mortality_age_func(mortality_age2017, years_list2017, men_range2017, women_range2017)
mortality_age2017.iloc[0, 0] = 0
mortality_age2017.iloc[16, 0] = 0

In [64]:
# Демонстрация измененного датафрейма

mortality_age2017.head()

,возраст,2005,2010,2013,2014,2015,2016,пол
0,0,"12,5","8,6","9,0","8,1","7,3","6,6",мужчины
1,1-4,"0,8","0,6","0,5","0,5","0,4","0,4",мужчины
2,5-9,"0,5","0,4","0,3","0,3","0,3","0,2",мужчины
3,10-14,"0,5","0,4","0,4","0,4","0,3","0,3",мужчины
4,15-19,"1,6","1,2","1,1","1,1","1,0","0,9",мужчины


In [65]:
# Запись исходного датафрейма в переменную

mortality_age2019 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2019\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.5. возрастные коэффициенты смертности (умершие на 1000 человек населения соответствующей возрастной группы).csv")
mortality_age2019.head()

,Unnamed: 0,2005,2010,2015,2016,2017,2018
0,Мужчины,"18,7","15,9","14,2","14,0","13,4","13,4"
1,из них:,NaN,NaN,NaN,NaN,NaN,NaN
2,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN,NaN
3,01),"12,5","8,6","7,3","6,6","6,1","5,6"
4,1 – 4,"0,8","0,6","0,4","0,4","0,4","0,3"


In [66]:
# Применение функции для обработки исходного датафрейма

years_list2019 = [2005, 2010, 2015, 2016, 2017, 2018]
men_range2019 = slice(3, 19)
women_range2019 = slice(24, 40)

mortality_age2019 = mortality_age_func(mortality_age2019, years_list2019, men_range2019, women_range2019)
mortality_age2019.iloc[0, 0] = 0
mortality_age2019.iloc[16, 0] = 0

In [67]:
# Демонстрация измененного датафрейма

mortality_age2019.head()

,возраст,2005,2010,2015,2016,2017,2018,пол
0,0,"12,5","8,6","7,3","6,6","6,1","5,6",мужчины
1,1 – 4,"0,8","0,6","0,4","0,4","0,4","0,3",мужчины
2,5 – 9,"0,5","0,4","0,3","0,2","0,2","0,2",мужчины
3,10 – 14,"0,5","0,4","0,3","0,3","0,3","0,3",мужчины
4,15 – 19,"1,6","1,2","1,0","0,9","0,9","0,8",мужчины


In [68]:
# Запись исходного датафрейма в переменную

mortality_age2021 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2021\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\1.5. возрастные коэффициенты смертности (умершие на 1000 человек населения соответствующей возрастной группы).csv")
mortality_age2021.head()

,Unnamed: 0,2005,2010,2015,2018,2019,2020
0,Мужчины,"18,7","15,9","14,2","13,4","13,2","15,7"
1,из них:,NaN,NaN,NaN,NaN,NaN,NaN
2,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN,NaN
3,01),"12,5","8,6","7,3","5,6","5,3","5,0"
4,1 – 4,"0,8","0,6","0,4","0,3","0,3","0,3"


In [69]:
# Применение функции для обработки исходного датафрейма

years_list2021 = [2005, 2010, 2015, 2018, 2019, 2020]
men_range2021 = slice(3, 19)
women_range2021 = slice(24, 40)

mortality_age2021 = mortality_age_func(mortality_age2021, years_list2021, men_range2021, women_range2021)
mortality_age2021.iloc[0, 0] = 0
mortality_age2021.iloc[16, 0] = 0

In [70]:
# Демонстрация измененного датафрейма

mortality_age2021.head()

,возраст,2005,2010,2015,2018,2019,2020,пол
0,0,"12,5","8,6","7,3","5,6","5,3","5,0",мужчины
1,1 – 4,"0,8","0,6","0,4","0,3","0,3","0,3",мужчины
2,5 – 9,"0,5","0,4","0,3","0,2","0,2","0,2",мужчины
3,10 – 14,"0,5","0,4","0,3","0,3","0,3","0,3",мужчины
4,15 – 19,"1,6","1,2","1,0","0,8","0,8","0,8",мужчины


In [71]:
# Запись исходного датафрейма в переменную

mortality_age2023 = pd.read_excel("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2023\\Zdrav_01-2023.xlsx", sheet_name='1.5')
mortality_age2023.head()

,1.5. ВОЗРАСТНЫЕ КОЭФФИЦИЕНТЫ СМЕРТНОСТИ1)\n(умершие на 1000 человек населения соответствующей возрастной группы),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,2010.0,2015.0,2019.0,2020.0,2021.0,2022.0
1,Мужчины,15.9,14.2,13.2,15.7,17.2,14.0
2,из них:,NaN,NaN,NaN,NaN,NaN,NaN
3,"в возрасте, лет",NaN,NaN,NaN,NaN,NaN,NaN
4,02),8.6,7.3,5.3,5.0,5.3,4.9


In [72]:
# Применение функции для обработки исходного датафрейма

years_list2023 = [2010, 2015, 2019, 2020, 2021, 2022]
men_range2023 = slice(4, 20)
women_range2023 = slice(25, 41)

mortality_age2023 = mortality_age_func(mortality_age2023, years_list2023, men_range2023, women_range2023)
mortality_age2023.iloc[0, 0] = 0
mortality_age2023.iloc[16, 0] = 0

In [73]:
# Демонстрация измененного датафрейма

mortality_age2023.head()

,возраст,2010,2015,2019,2020,2021,2022,пол
0,0,8.6,7.3,5.3,5.0,5.3,4.9,мужчины
1,1 – 4,0.6,0.4,0.3,0.3,0.3,0.3,мужчины
2,5 – 9,0.4,0.3,0.2,0.2,0.2,0.2,мужчины
3,10 – 14,0.4,0.3,0.3,0.3,0.3,0.3,мужчины
4,15 – 19,1.2,1.0,0.8,0.8,0.8,0.8,мужчины


---
Объединяем все измененные датасеты в один.

Удаляем дублирующиеся годы, причем оставляя записи из более "поздних" датасетов (предполагается, что в них содержатся более актуальные данные).

---

In [74]:
# Объединение датасетов, удаление дублирующихся годов

mortality_age_combined = (
    pd.concat([mortality_age2005, mortality_age2007, mortality_age2009, mortality_age2013, mortality_age2015, mortality_age2017, mortality_age2019, mortality_age2021, mortality_age2023], axis=1)
    .T
    .reset_index()
    .rename(columns={'index':'год'})
    .drop_duplicates(subset='год', keep = 'last')
    .sort_values(by='год', key = lambda col: col.astype(str))
    .T
)

In [75]:
# Обработка перечня столбцов 

mortality_age_combined.columns = mortality_age_combined.iloc[0]
mortality_age_combined = mortality_age_combined.drop(mortality_age_combined.index[0])

In [76]:
# Изменение порядка столбцов в датафреме

mortality_age_combined = mortality_age_combined.set_index('возраст').reset_index()
columns_order = ['пол'] + [col for col in mortality_age_combined.columns if col != 'пол'] 

mortality_age_combined = mortality_age_combined[columns_order]

In [77]:
# Замена символов разделителей, замена типа данных в столбцах (кроме столбцов пола и возраста)

mortality_age_combined = mortality_age_combined.replace(to_replace=',', value='.', regex=True)
mortality_age_combined = mortality_age_combined.replace(to_replace='25 –29', value='25 – 29')

columns_to_convert = [col for col in mortality_age_combined.columns if col not in ['пол', 'возраст']]
mortality_age_combined[columns_to_convert] = mortality_age_combined[columns_to_convert].astype(float)

In [78]:
# Удаление лишних столбцов (рассматривается период с 2001 года)

mortality_age_combined = mortality_age_combined.drop(columns=[1990, 1995, 2000])

In [79]:
mortality_age_combined.info()
mortality_age_combined.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 23 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   пол      32 non-null     object 
 1   возраст  32 non-null     object 
 2   2001     32 non-null     float64
 3   2002     32 non-null     float64
 4   2003     32 non-null     float64
 5   2004     32 non-null     float64
 6   2005     32 non-null     float64
 7   2006     32 non-null     float64
 8   2007     32 non-null     float64
 9   2008     32 non-null     float64
 10  2010     32 non-null     float64
 11  2011     32 non-null     float64
 12  2012     32 non-null     float64
 13  2013     32 non-null     float64
 14  2014     32 non-null     float64
 15  2015     32 non-null     float64
 16  2016     32 non-null     float64
 17  2017     32 non-null     float64
 18  2018     32 non-null     float64
 19  2019     32 non-null     float64
 20  2020     32 non-null     float64
 21  2021     32 non-nu

год,пол,возраст,2001,2002,2003,2004,2005,2006,2007,2008,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,мужчины,0,16.9,15.3,13.8,13.1,12.5,11.4,10.5,9.5,8.6,8.3,9.5,9.0,8.1,7.3,6.6,6.1,5.6,5.3,5.0,5.3,4.9
1,мужчины,1 – 4,1.1,1.0,1.0,0.8,0.8,0.8,0.7,0.7,0.6,0.6,0.5,0.5,0.5,0.4,0.4,0.4,0.3,0.3,0.3,0.3,0.3
2,мужчины,5 – 9,0.6,0.5,0.5,0.5,0.5,0.4,0.4,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.2,0.2,0.2,0.2,0.2,0.2,0.2
3,мужчины,10 – 14,0.6,0.6,0.6,0.6,0.5,0.5,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3
4,мужчины,15 – 19,1.9,1.8,1.7,1.7,1.6,1.6,1.5,1.5,1.2,1.2,1.1,1.1,1.1,1.0,0.9,0.9,0.8,0.8,0.8,0.8,0.8


In [80]:
# Преобразуем датафрейм в длинный для удобства формирования дашборда 

mortality_age_melted = mortality_age_combined.melt(id_vars=['пол', 'возраст'], var_name='год', value_name='значение')
mortality_age_melted.head(10)

,пол,возраст,год,значение
0,мужчины,0,2001,16.9
1,мужчины,1 – 4,2001,1.1
2,мужчины,5 – 9,2001,0.6
3,мужчины,10 – 14,2001,0.6
4,мужчины,15 – 19,2001,1.9
5,мужчины,20 – 24,2001,4.4
6,мужчины,25 – 29,2001,5.8
7,мужчины,30 – 34,2001,7.1
8,мужчины,35 – 39,2001,9.4
9,мужчины,40 – 44,2001,13.1


In [81]:
# Запись скомпилированного датафрейма в файл

mortality_age_combined.to_csv('mortality_age_combined.csv', encoding='utf-8')

mortality_age_melted.to_csv('mortality_age_melted.csv', encoding='utf-8')

### 1.3. Перинатальная смертность

---
Сведения о перинатальной смертности по годам и по полу (с разделением на мертворожденных и умерших в первые 7 дней жизни) содержатся в нескольких таблицах (перинатальная смертность).

---

In [82]:
# Создание функции для обработки датафреймов, содержащих сведения о перинатальной смертности. Функция принимает на вход исходный датафрейм и список индексов строк, которые нужно удалить из итогового датафрейма. Кроме того, функция создает и заполняет столбец с годом и меняет порядок столбцов


def perinatality_func(df, rows_to_drop):
    df.columns = ['пол', 'умершие, всего', 'мертворожденные', 'в возрасте до 7 дней', 'коэффициент смертности, всего', 'коэффициент смертности, мертворожденные', 'коэффициент смертности, в возрасте до 7 дней']
    def is_year(value):
        try:
            int(value)
            return True
        except ValueError:
            return False
    df['год'] = df['пол'].apply(lambda x: x if is_year(x) else None)
    df['год'] = df['год'].ffill()
    columns_order = ['пол'] + ['год'] + [col for col in perinatality_2001.columns if col not in ['пол', 'год']]
    df = df[columns_order]
    df = df.drop(rows_to_drop)
    return df

In [83]:
# Запись исходного датафрейма в переменную

perinatality_2001 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2001\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\перинатальная смертность.csv")
perinatality_2001

,0,1,2,3,4,5,6
0,NaN,"Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек",Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...
1,NaN,всего,в том числе,в том числе,все умершие в перинатальном периоде,меpтво- pожденные,умеpшие в возрасте до 7 дней
2,NaN,всего,меpтво- pожденные,умеpшие в возрасте до 7 дней,все умершие в перинатальном периоде,меpтво- pожденные,умеpшие в возрасте до 7 дней
3,1990,NaN,NaN,NaN,NaN,NaN,NaN
4,Мальчики,20726,9861,10865,20.10,9.56,10.54
5,Девочки,15273,8304,6969,15.65,8.51,7.14
6,Оба пола,35999,18165,17834,17.94,9.05,8.89
7,1995,NaN,NaN,NaN,NaN,NaN,NaN
8,Мальчики,12459,5498,6961,17.66,7.79,9.87
9,Девочки,9300,4661,4639,13.92,6.98,6.94


In [84]:
# Применение функции для обработки исходного датафрейма

rows_to_drop2001 = [0, 1, 2, 3, 7, 11, 15, 19, 23, 27]
perinatality_2001 = perinatality_func(perinatality_2001, rows_to_drop2001)

In [85]:
# Для единообразия во всех таблицах переименовываем категорию

perinatality_2001 = perinatality_2001.replace(to_replace='Оба пола', value='Мальчики и девочки')

In [86]:
# Демонстрация измененного датафрейма

perinatality_2001

,пол,год,"умершие, всего",мертворожденные,в возрасте до 7 дней,"коэффициент смертности, всего","коэффициент смертности, мертворожденные","коэффициент смертности, в возрасте до 7 дней"
4,Мальчики,1990,20726,9861,10865,20.10,9.56,10.54
5,Девочки,1990,15273,8304,6969,15.65,8.51,7.14
6,Мальчики и девочки,1990,35999,18165,17834,17.94,9.05,8.89
8,Мальчики,1995,12459,5498,6961,17.66,7.79,9.87
9,Девочки,1995,9300,4661,4639,13.92,6.98,6.94
10,Мальчики и девочки,1995,21759,10159,11600,15.83,7.39,8.44
12,Мальчики,1996,11965,5492,6473,17.68,8.12,9.56
13,Девочки,1996,8913,4753,4160,13.97,7.45,6.52
14,Мальчики и девочки,1996,20878,10245,10633,15.88,7.79,8.09
16,Мальчики,1997,11347,5434,5913,17.36,8.31,9.05


In [87]:
# Запись исходного датафрейма в переменную

perinatality_2005 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2005\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\перинатальная смертность.csv")
perinatality_2005.head()

,0,1,2,3,4,5,6
0,NaN,"Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек",Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...
1,NaN,всего,в том числе,в том числе,всего,меpтво- pожденные,умеpшие в возрасте до 7 дней
2,NaN,всего,меpтво- pожденные,умеpшие в возрасте до 7 дней,всего,меpтво- pожденные,умеpшие в возрасте до 7 дней
3,1990,NaN,NaN,NaN,NaN,NaN,NaN
4,Мальчики,20726,9861,10865,20.10,9.56,10.54


In [88]:
# Применение функции для обработки исходного датафрейма

rows_to_drop2005 = [0, 1, 2, 3, 7, 11, 15, 19, 23, 27]
perinatality_2005 = perinatality_func(perinatality_2005, rows_to_drop2005)

In [89]:
# Демонстрация измененного датафрейма

perinatality_2005.head()

,пол,год,"умершие, всего",мертворожденные,в возрасте до 7 дней,"коэффициент смертности, всего","коэффициент смертности, мертворожденные","коэффициент смертности, в возрасте до 7 дней"
4,Мальчики,1990,20726,9861,10865,20.10,9.56,10.54
5,Девочки,1990,15273,8304,6969,15.65,8.51,7.14
6,Мальчики и девочки,1990,35999,18165,17834,17.94,9.05,8.89
8,Мальчики,1995,12459,5498,6961,17.66,7.79,9.87
9,Девочки,1995,9300,4661,4639,13.92,6.98,6.94


In [90]:
# Запись исходного датафрейма в переменную

perinatality_2007 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2007\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\перинатальная смертность.csv")
perinatality_2007.head()

,0,1,2,3,4,5,6
0,NaN,"Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек",Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...
1,NaN,всего,в том числе,в том числе,всего,меpтво- pожден- ные,умеpшие в возрасте до 7 дней
2,NaN,всего,меpтво- pожден- ные,умеpшие в возрасте до 7 дней,всего,меpтво- pожден- ные,умеpшие в возрасте до 7 дней
3,1990,NaN,NaN,NaN,NaN,NaN,NaN
4,Мальчики,20726,9861,10865,20.10,9.56,10.54


In [91]:
# Применение функции для обработки исходного датафрейма

rows_to_drop2007 = [0, 1, 2, 3, 7, 11, 15, 19]
perinatality_2007 = perinatality_func(perinatality_2007, rows_to_drop2007)

In [92]:
# Демонстрация измененного датафрейма

perinatality_2007.head()

,пол,год,"умершие, всего",мертворожденные,в возрасте до 7 дней,"коэффициент смертности, всего","коэффициент смертности, мертворожденные","коэффициент смертности, в возрасте до 7 дней"
4,Мальчики,1990,20726,9861,10865,20.10,9.56,10.54
5,Девочки,1990,15273,8304,6969,15.65,8.51,7.14
6,Мальчики и девочки,1990,35999,18165,17834,17.94,9.05,8.89
8,Мальчики,1995,12459,5498,6961,17.66,7.79,9.87
9,Девочки,1995,9300,4661,4639,13.92,6.98,6.94


In [93]:
# Запись исходного датафрейма в переменную

perinatality_2009 = pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2009\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\перинатальная смертность.csv")
perinatality_2009.head()

,0,1,2,3,4,5,6
0,NaN,"Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек","Умершие в перинатальном периоде, человек",Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...,Коэффициент перинатальной смертности (на 1000 ...
1,NaN,всего,в том числе,в том числе,всего,меpтво- pожденные,умеpшие в возрасте до 7 дней
2,NaN,всего,меpтво- pожденные,умеpшие в возрасте до 7 дней,всего,меpтво- pожденные,умеpшие в возрасте до 7 дней
3,1995,NaN,NaN,NaN,NaN,NaN,NaN
4,Мальчики,12459,5498,6961,17.66,7.79,9.87


In [94]:
# Применение функции для обработки исходного датафрейма

rows_to_drop2009 = [0, 1, 2, 3, 7, 11, 15, 19]
perinatality_2009 = perinatality_func(perinatality_2009, rows_to_drop2009)

In [95]:
# Демонстрация измененного датафрейма

perinatality_2009.head()

,пол,год,"умершие, всего",мертворожденные,в возрасте до 7 дней,"коэффициент смертности, всего","коэффициент смертности, мертворожденные","коэффициент смертности, в возрасте до 7 дней"
4,Мальчики,1995,12459,5498,6961,17.66,7.79,9.87
5,Девочки,1995,9300,4661,4639,13.92,6.98,6.94
6,Мальчики и девочки,1995,21759,10159,11600,15.83,7.39,8.44
8,Мальчики,2000,9646,4643,5003,14.66,7.06,7.60
9,Девочки,2000,7164,3851,3313,11.61,6.24,5.37


In [96]:
# Запись исходного датафрейма в переменную

perinatality_2023 = pd.read_excel("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2023\\Zdrav_01-2023.xlsx", sheet_name='1.13')
perinatality_2023.head()

,1.13. ПЕРИНАТАЛЬНАЯ СМЕРТНОСТЬ,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,"Умершие в перинатальном периоде, человек",NaN,NaN,"Коэффициент перинатальной смертности, на 1000 ...",NaN,NaN
1,NaN,всего,в том числе,NaN,всего,в том числе,NaN
2,NaN,NaN,меpтво-pожденные,умеpшие \nв возрасте \nдо 7 дней,NaN,меpтво-pожденные,умеpшие \nв возрасте \nдо 7 дней
3,2010,NaN,NaN,NaN,NaN,NaN,NaN
4,Мальчики,7 315,4 449,2 866,7.92,4.81,3.11


In [97]:
# Применение функции для обработки исходного датафрейма

rows_to_drop2023 = [0, 1, 2, 3, 7, 11, 15, 19, 23]
perinatality_2023 = perinatality_func(perinatality_2023, rows_to_drop2023)

In [98]:
# Демонстрация измененного датафрейма

perinatality_2023.head()

,пол,год,"умершие, всего",мертворожденные,в возрасте до 7 дней,"коэффициент смертности, всего","коэффициент смертности, мертворожденные","коэффициент смертности, в возрасте до 7 дней"
4,Мальчики,2010.0,7 315,4 449,2 866,7.92,4.81,3.11
5,Девочки,2010.0,5 933,3 851,2 082,6.8,4.41,2.39
6,Мальчики и девочки,2010.0,13 248,8 300,4 948,7.37,4.62,2.75
8,Мальчики,2015.0,8 906,6 110,2 796,8.87,6.08,2.8
9,Девочки,2015.0,7 267,5 343,1 924,7.67,5.64,2.04


---

Объединяем все измененные датасеты в один.

---

In [99]:
# Объединение датасетов по перинатальной смертности и 

perinatality_combined = pd.concat([perinatality_2001, perinatality_2005, perinatality_2007, perinatality_2009, perinatality_2023])

In [100]:
# Приводим столбцы датафрейма к нужным типам данных

perinatality_combined['год'] = perinatality_combined['год'].astype(int)

perinatality_combined[['умершие, всего', 'мертворожденные', 'в возрасте до 7 дней']] = perinatality_combined[['умершие, всего', 'мертворожденные', 'в возрасте до 7 дней']].apply(lambda x: x.astype(str).str.replace('\xa0', '', regex=True).astype(int), axis=0)

perinatality_combined[['коэффициент смертности, всего', 'коэффициент смертности, мертворожденные', 'коэффициент смертности, в возрасте до 7 дней']] = perinatality_combined[['коэффициент смертности, всего', 'коэффициент смертности, мертворожденные', 'коэффициент смертности, в возрасте до 7 дней']].apply(lambda x: x.astype(float), axis=0)

In [101]:
# Удаление дубликатов, очистка датафрейма от значений до 2001 года

perinatality_combined = perinatality_combined.drop_duplicates(subset=['год', 'пол'], keep = 'last').query('год >= 2001').sort_values(by=['год', 'пол']).reset_index(drop=True)

In [102]:
perinatality_combined.info()
perinatality_combined.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 8 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   пол                                           42 non-null     object 
 1   год                                           42 non-null     int32  
 2   умершие, всего                                42 non-null     int32  
 3   мертворожденные                               42 non-null     int32  
 4   в возрасте до 7 дней                          42 non-null     int32  
 5   коэффициент смертности, всего                 42 non-null     float64
 6   коэффициент смертности, мертворожденные       42 non-null     float64
 7   коэффициент смертности, в возрасте до 7 дней  42 non-null     float64
dtypes: float64(3), int32(4), object(1)
memory usage: 2.1+ KB


,пол,год,"умершие, всего",мертворожденные,в возрасте до 7 дней,"коэффициент смертности, всего","коэффициент смертности, мертворожденные","коэффициент смертности, в возрасте до 7 дней"
0,Девочки,2001,7159,3986,3173,11.20,6.23,4.97
1,Мальчики,2001,9744,4725,5019,14.31,6.94,7.37
2,Мальчики и девочки,2001,16903,8711,8192,12.80,6.60,6.20
3,Девочки,2002,7324,4097,3227,10.75,6.01,4.74
4,Мальчики,2002,9661,4901,4760,13.34,6.77,6.57


In [103]:
# Запись скомпилированного датафрейма в файл

perinatality_combined.to_csv('perinatality_combined.csv', encoding='utf-8')

### 1.4. Статистическая проверка гипотез по перинатальной смертности

---

Проверим статистическим методом гипотезу о том, что за период с 2001 по 2023 годы соотношение мертворожденных мужского и женского пола не имеет статистически значимых отличий, в то время как соотношение перинатально умерших (первые 168 часов, 7 суток после родов) мальчиков и девочек всегда (из года в год) выше у мальчиков.

Используем Z-тест для проверки гипотезы о равенстве долей (пропорций), то есть для каждого года сравним долю мертворожденных мальчиков от общего количества умерших в перинатальном периоде мальчиков, и долю мертворожденных девочек от общего количества умерших в перинатальном периоде девочек.
В связи с большим количеством измерений применим простую поправку Бонферрони, разделив уровень статистической значимости alpha на количество измерений. 

**Нулевая гипотеза:** нет оснований считать доли разными.

**Альтернативная гипотеза:** между долями есть статистически значимая разница.

---

In [104]:
for year in perinatality_combined['год'].unique():
    successes1 = np.array([perinatality_combined.query("год == @year & пол == 'Девочки'")['мертворожденные'],\
                         perinatality_combined.query("год == @year & пол == 'Мальчики'")['мертворожденные']])
    nobs1 = np.array([perinatality_combined.query("год == @year & пол == 'Девочки'")['умершие, всего'],\
                         perinatality_combined.query("год == @year & пол == 'Мальчики'")['умершие, всего']])
    
    z_stat, p_value = proportions_ztest(count=successes1, nobs=nobs1)
    
    alpha = 0.05 / len(perinatality_combined['год'].unique())

    print(f'Год: {year}, p-value: {p_value}')
    
    if successes1[0]/nobs1[0] > successes1[1]/nobs1[1]:
        print('Доля мертворожденных девочек больше')
    elif successes1[1]/nobs1[1] > successes1[0]/nobs1[0]:
        print('Доля мертворожденных мальчиков больше')
    else: print('Доли равны')
        
    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

Год: 2001, p-value: [2.50917304e-20]
Доля мертворожденных девочек больше
Отвергаем нулевую гипотезу: между долями есть значимая разница
Год: 2002, p-value: [1.61789379e-11]
Доля мертворожденных девочек больше
Отвергаем нулевую гипотезу: между долями есть значимая разница
Год: 2003, p-value: [6.6047479e-12]
Доля мертворожденных девочек больше
Отвергаем нулевую гипотезу: между долями есть значимая разница
Год: 2004, p-value: [2.27690225e-16]
Доля мертворожденных девочек больше
Отвергаем нулевую гипотезу: между долями есть значимая разница
Год: 2005, p-value: [7.16273873e-16]
Доля мертворожденных девочек больше
Отвергаем нулевую гипотезу: между долями есть значимая разница
Год: 2006, p-value: [6.63749517e-13]
Доля мертворожденных девочек больше
Отвергаем нулевую гипотезу: между долями есть значимая разница
Год: 2007, p-value: [1.08282295e-11]
Доля мертворожденных девочек больше
Отвергаем нулевую гипотезу: между долями есть значимая разница
Год: 2008, p-value: [7.36794292e-07]
Доля мертвор

---

Как видим, предположение о том, что соотношение мертворожденных мужского и женского пола не имеет статистически значимых отличий, не подтвердилось. В каждом году:

- доля мертворожденных девочек была больше, чем доля мертворожденных мальчиков;
- разница между этими долями была статистически значима; причем p-значение за каждый год намного меньше уровня статистической значимости, что говорит о том, что разница между долями статистически очень значима.

Соответственно, при сравнении долей мальчиков и девочек, умерших в возрасте до 7 дней, соотношение будет обратным - доля мальчиков за каждый год будет выше, чем доля девочек, - и разница между этими долями точно так же будет статистически очень значима (Z-тест выдаст такие же p-значения, и проводить его второй раз смысла нет).

---

### 1.5. Возрастные коэффициенты рождаемости

---

Сведения о возрастных коэффициентах рождаемости (число детей, родившихся живыми, на 1000 женщин, принадлежащих к одной возрастной группе) по годам содержатся в нескольких таблицах. К сожалению, в наборе таблиц отсутствуют сведения о возрастных коэффициентах рождаемости за некоторые годы.

---

In [105]:
# Запись исходного датафрейма в переменную

childbirth_age2005 =  pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2005\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\возрастные коэффициенты рождаемости.csv")

In [106]:
childbirth_age2005

,0,1,2,3,4,5,6,7,8,9
0,Годы,"Родившиеся живыми на 1000 женщин в возрасте, лет","Родившиеся живыми на 1000 женщин в возрасте, лет","Родившиеся живыми на 1000 женщин в возрасте, лет","Родившиеся живыми на 1000 женщин в возрасте, лет","Родившиеся живыми на 1000 женщин в возрасте, лет","Родившиеся живыми на 1000 женщин в возрасте, лет","Родившиеся живыми на 1000 женщин в возрасте, лет","Родившиеся живыми на 1000 женщин в возрасте, лет",Суммарный коэффициент рождаемости
1,Годы,моложе 201),20-24,25-29,30-34,35-39,40-44,45-49,15-49 2),Суммарный коэффициент рождаемости
2,1990,55.0,156.5,93.1,48.2,19.4,4.2,0.1,55.2,1.892
3,1995,44.8,112.7,66.6,29.5,10.6,2.2,0.1,35.9,1.337
4,2000,27.3,93.2,67.0,35.0,11.8,2.4,0.1,32.0,1.190
5,2001,27.3,93.1,70.2,38.0,12.9,2.4,0.1,33.1,1.223
6,2002,27.4,95.7,75.1,41.7,14.7,2.6,0.1,35.2,1.286
7,2003,27.6,95.1,78.3,44.1,16.0,2.7,0.1,36.5,1.319
8,2004,28.2,93.4,80.2,45.9,17.6,2.9,0.1,37.7,1.340


In [107]:
# Обработка строк, столбцов

childbirth_age2005 = childbirth_age2005.drop(childbirth_age2005.index[[0, 1]]).drop(columns=['8']).reset_index(drop=True)
childbirth_age2005.columns = ['год', 'моложе 20', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', 'cуммарный коэффициент рождаемости']

In [108]:
childbirth_age2005

,год,моложе 20,20-24,25-29,30-34,35-39,40-44,45-49,cуммарный коэффициент рождаемости
0,1990,55.0,156.5,93.1,48.2,19.4,4.2,0.1,1.892
1,1995,44.8,112.7,66.6,29.5,10.6,2.2,0.1,1.337
2,2000,27.3,93.2,67.0,35.0,11.8,2.4,0.1,1.190
3,2001,27.3,93.1,70.2,38.0,12.9,2.4,0.1,1.223
4,2002,27.4,95.7,75.1,41.7,14.7,2.6,0.1,1.286
5,2003,27.6,95.1,78.3,44.1,16.0,2.7,0.1,1.319
6,2004,28.2,93.4,80.2,45.9,17.6,2.9,0.1,1.340


In [109]:
# Запись исходного датафрейма в переменную

childbirth_age2007 =  pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2007\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\возрастные коэффициенты рождаемости.csv")

In [110]:
# Обработка строк, столбцов

childbirth_age2007 = childbirth_age2007.drop(childbirth_age2007.index[[0, 1]]).drop(columns=['8']).reset_index(drop=True)
childbirth_age2007.columns = ['год', 'моложе 20', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', 'cуммарный коэффициент рождаемости']

In [111]:
childbirth_age2007

,год,моложе 20,20-24,25-29,30-34,35-39,40-44,45-49,cуммарный коэффициент рождаемости
0,1990,55.0,156.5,93.1,48.2,19.4,4.2,0.1,1.892
1,1995,44.8,112.7,66.5,29.5,10.6,2.2,0.1,1.337
2,2000,27.4,93.6,67.3,35.2,11.8,2.4,0.1,1.195
3,2005,27.5,86.8,77.9,45.5,17.8,3.0,0.2,1.287
4,2006,28.6,85.8,78.2,46.8,18.7,3.1,0.1,1.296


In [112]:
# Запись исходного датафрейма в переменную

childbirth_age2009 =  pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2009\\ДЕМОГРАФИЧЕСКАЯ СИТУАЦИЯ\\возрастные коэффициенты рождаемости.csv")

In [113]:
# Обработка строк, столбцов

childbirth_age2009 = childbirth_age2009.drop(childbirth_age2009.index[[0, 1]]).drop(columns=['8']).reset_index(drop=True)
childbirth_age2009.columns = ['год', 'моложе 20', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', 'cуммарный коэффициент рождаемости']

In [114]:
childbirth_age2009

,год,моложе 20,20-24,25-29,30-34,35-39,40-44,45-49,cуммарный коэффициент рождаемости
0,1995,44.8,112.7,66.5,29.5,10.6,2.2,0.1,1.337
1,2000,27.4,93.6,67.3,35.2,11.8,2.4,0.1,1.195
2,2005,27.5,86.8,77.9,45.5,17.8,3.0,0.2,1.287
3,2007,29.1,87.5,86.3,54.5,22.8,3.8,0.2,1.406
4,2008,30.5,89.8,91.0,60.4,25.9,4.6,0.2,1.494


In [115]:
# Запись исходного датафрейма в переменную

childbirth_age2023 = pd.read_excel("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\2023\\Zdrav_01-2023.xlsx", sheet_name='1.4')

In [116]:
childbirth_age2023

,1.4. ВОЗРАСТНЫЕ КОЭФФИЦИЕНТЫ РОЖДАЕМОСТИ1),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Годы,"Родившиеся живыми на 1000 женщин в возрасте, лет",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Суммарный
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,коэффициент
2,NaN,моложе 202),20 – 24,25 – 29,30 – 34,35 – 39,40 – 44,45 – 49,15 – 493),рождаемости
3,2010,27,87.5,99.2,67.3,30,5.9,0.3,47.8,1.567
4,2015,24,90,112.6,83,39.8,8.3,0.4,54.6,1.777
5,2019,14.7,74.8,91.2,71.6,38.7,8.9,0.5,42.8,1.504
6,2020,14.1,73.6,92.6,70.8,39.2,9.2,0.6,41.8,1.505
7,2021,13.5,70.6,94.8,71,40.1,9.5,0.6,41,1.505
8,2022,13.4,65.7,90.1,65.4,37.6,9.3,0.6,37.8,1.416
9,"1) Данные за 2015, 2019-2021 годы без учета и...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# Обработка строк, столбцов

childbirth_age2023 = childbirth_age2023.drop(childbirth_age2023.index[[0, 1, 2, 9, 10, 11]]).drop(columns=['Unnamed: 8']).reset_index(drop=True)
childbirth_age2023.columns = ['год', 'моложе 20', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', 'cуммарный коэффициент рождаемости']

In [118]:
childbirth_age2023

,год,моложе 20,20-24,25-29,30-34,35-39,40-44,45-49,cуммарный коэффициент рождаемости
0,2010,27,87.5,99.2,67.3,30,5.9,0.3,1.567
1,2015,24,90,112.6,83,39.8,8.3,0.4,1.777
2,2019,14.7,74.8,91.2,71.6,38.7,8.9,0.5,1.504
3,2020,14.1,73.6,92.6,70.8,39.2,9.2,0.6,1.505
4,2021,13.5,70.6,94.8,71,40.1,9.5,0.6,1.505
5,2022,13.4,65.7,90.1,65.4,37.6,9.3,0.6,1.416


In [119]:
# Объединение датафреймов, обработка типов данных в столбцах

childbirth_age_combined = pd.concat([childbirth_age2005, childbirth_age2007, childbirth_age2009, childbirth_age2023], axis=0)
childbirth_age_combined = childbirth_age_combined.astype(float)
childbirth_age_combined['год'] = childbirth_age_combined['год'].astype(int)

In [120]:
# Удаление дубликатов в годах (исходим из того, что более поздние данные - более актуальные)

childbirth_age_combined = (
    childbirth_age_combined
    .drop_duplicates(subset='год', keep='last')
    .sort_values(by='год')
    .query('год >= 2001')
    .reset_index(drop=True)
)

In [121]:
# Запись в файл

childbirth_age_combined.to_csv('childbirth_age_combined.csv', encoding='utf-8')

In [122]:
# Создание дополнительного датафрейма на основе объединенного, с другой структурой столбцов

childbirth_age_combined_melted = childbirth_age_combined.melt(id_vars=['год', 'cуммарный коэффициент рождаемости'], var_name='возрастная категория', value_name='значение')

In [123]:
# Запись в файл

childbirth_age_combined_melted.to_csv('childbirth_age_combined_melted.csv', encoding='utf-8')

In [124]:
childbirth_age_combined_melted

,год,cуммарный коэффициент рождаемости,возрастная категория,значение
0,2001,1.223,моложе 20,27.3
1,2002,1.286,моложе 20,27.4
2,2003,1.319,моложе 20,27.6
3,2004,1.340,моложе 20,28.2
4,2005,1.287,моложе 20,27.5
...,...,...,...,...
93,2015,1.777,45-49,0.4
94,2019,1.504,45-49,0.5
95,2020,1.505,45-49,0.6
96,2021,1.505,45-49,0.6


### 1.6. Альтернативная выгрузка по численности и возрастным группам населения

---

В связи с тем, что в изначальном наборе таблиц отсутствовали сведения о численности населения по регионам и возрастным группам по субъектам РФ, я решил сделать альтернативную выгрузку с портала showdata.gks.ru (витрина статистических данных), чтобы посмотреть численность постоянного населения в разрезе возрастных групп по регионам. К сожалению, обнаружилось, что данные о численности постоянного населения РФ в этой выгрузке лишь в половине случаев совпадали с данными, которые есть в исходном наборе таблиц. Поэтому я строил по выгрузке отдельные графики на дашборде.

---

In [125]:
# Запись загруженного датафрейма в переменную

headcount_reg =  pd.read_csv("C:\\Users\\pavli\\Documents\\РосстатЗдрав\\WEBmedSTAT\\Численность_постоянного_населения_России_по_возрасту.csv", sep=";")

In [126]:
headcount_reg.head()

,Регион,Возрастная категория,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Алтайский край,Всего,2621050,2602595,2571987,2539430,2503510,2473024,2453455,2438910,2430765,2417358,2396786,2377980,2359540,2343388,2325025,2303606,2277683,2250096,2224119,2193007,2154932,2130950,2115308
1,Алтайский край,Старше трудоспособного,527084,521517,515581,512627,510396,514515,521097,528938,540485,552431,562230,571582,581893,591912,602159,609575,615867,619930,590822,587313,548399,550938,527851
2,Алтайский край,В трудоспособном возрасте,1608270,1616992,1613803,1602645,1583732,1562232,1541647,1518751,1493828,1464683,1427830,1390691,1353382,1319796,1285036,1253395,1219934,1190012,1198139,1175912,1182444,1162062,1179601
3,Алтайский край,Моложе трудоспособного,485696,464086,442603,424158,409382,396277,390711,391221,396452,400244,406726,415707,424265,431680,437830,440636,441882,440154,435158,429782,424089,417950,407856
4,Амурская область,Всего,911381,901044,887781,874018,861056,850502,844290,838869,834905,828660,820627,815163,808726,806524,801539,796801,792672,786641,782690,773692,763570,756198,750083


In [127]:
# Создание дополнительного датафрейма, с другой структурой столбцов

headcount_reg_melted = headcount_reg.melt(id_vars=['Регион', 'Возрастная категория'], var_name='Год', value_name='Значение')

In [128]:
headcount_reg_melted.head()

,Регион,Возрастная категория,Год,Значение
0,Алтайский край,Всего,2001,2621050
1,Алтайский край,Старше трудоспособного,2001,527084
2,Алтайский край,В трудоспособном возрасте,2001,1608270
3,Алтайский край,Моложе трудоспособного,2001,485696
4,Амурская область,Всего,2001,911381


In [129]:
# Запись в файл

headcount_reg_melted.to_csv('headcount_reg_melted.csv', encoding='utf-8')

In [130]:
# Создание дополнительного датафрейма, с другой структурой столбцов

headcount_reg_melted_pivot = headcount_reg_melted.pivot(index=['Регион', 'Год'], columns='Возрастная категория', values='Значение').reset_index(names=['Регион', 'Год'])


In [131]:
headcount_reg_melted_pivot.head()

Возрастная категория,Регион,Год,В трудоспособном возрасте,Всего,Моложе трудоспособного,Старше трудоспособного
0,Алтайский край,2001,1608270,2621050,485696,527084
1,Алтайский край,2002,1616992,2602595,464086,521517
2,Алтайский край,2003,1613803,2571987,442603,515581
3,Алтайский край,2004,1602645,2539430,424158,512627
4,Алтайский край,2005,1583732,2503510,409382,510396


In [132]:
# Запись в файл

headcount_reg_melted_pivot.to_csv('headcount_reg_melted_pivot.csv', encoding='utf-8')